import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import desc
from pyspark.sql.functions import when

create local SparkSession

In [ ]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

read csv with inferschema

In [ ]:
%%time
df = spark.read.options(header='True',inferSchema='True',delimiter=',') \
.csv("C:/Users/user/Downloads/ds_salaries.csv")
df.show()

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [ ]:
%%time
df = spark.read.options(header='True',inferSchema='True',delimiter=',') \
.csv("C:/Users/user/Downloads/ds_salaries.csv")

write schema of scv on screen

In [ ]:
df.printSchema()

create schema of this scv

In [ ]:
schema = StructType([ \
    StructField("id",IntegerType(),True), \
    StructField("work_year",IntegerType(),True), \
    StructField("experience_level",StringType(),True), \
    StructField("employment_type", StringType(), True), \
    StructField("job_title", StringType(), True), \
    StructField("salary", IntegerType(), True), \
    StructField("salary_currency", StringType(), True), \
    StructField("salary_in_usd", IntegerType(), True), \
    StructField("employee_residence", StringType(), True), \
    StructField("remote_ratio", IntegerType(), True), \
    StructField("company_location", StringType(), True), \
    StructField("company_size", StringType(), True) \
  ])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [ ]:
%%time
df2 = spark.read \
  .option("header", True) \
  .option("delimiter", ",") \
  .schema(schema) \
  .csv("C:/Users/user/Downloads/ds_salaries.csv")
df2.show()

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [ ]:
df2.printSchema()

now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [ ]:
df2.show()

print data in dataframe using display(df.toPandas())

In [ ]:
df2.toPandas()

create df_job_title that consists from all job_titles without duplicates

In [ ]:
df_job_title = df2.dropDuplicates(["job_title"])

print all rows from df_job_titles without truncating jobs

In [ ]:
df_job_title.show(truncate=False)

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [ ]:
df_analytic=df2.groupBy("job_title") \
    .agg(avg("salary_in_usd").alias("avg_salary"), \
         min("salary_in_usd").alias("min_salary"), \
         max("salary_in_usd").alias("max_salary"))

print all rows from df_analytic without trancating jobs

In [ ]:
df_analytic.show(truncate=False)

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [ ]:
windowSpec  = Window.orderBy(desc("avg_salary"))
df_analytic2=df_analytic.withColumn("row_id",row_number().over(windowSpec))

print all data from df_analytic

In [ ]:
df_analytic2.withColumn("row_id",row_number().over(windowSpec)).show(truncate=False)

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [ ]:
df_analytic2.select(col("row_id"), col("job_title"),
                   col("avg_salary"),col("min_salary"), col("max_salary"))

print df_analytic now

In [ ]:
df_analytic2.select(col("row_id"), col("job_title"),
                   col("avg_salary"),col("min_salary"), col("max_salary")).show(truncate=False)

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [ ]:
df_expe_lvl = df2
feature_group = ['experience_level']
data_max = df_expe_lvl.groupBy(feature_group).max("salary_in_usd")
data_max2=data_max.select("experience_level",col("max(salary_in_usd)").alias("biggest_salary"))
df_exp_lvl = df_expe_lvl.join(data_max2, feature_group).dropDuplicates(["experience_level"])

print here df_exp_lvl

In [ ]:
df_exp_lvl.toPandas()

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [ ]:
feature_group2=['experience_level']
data_maxy=df2.groupBy(feature_group2).max("salary_in_usd")
data_maxy2=data_maxy.select("experience_level",col("max(salary_in_usd)").alias("biggest_salary"))
df = df2.join(data_maxy2, feature_group2)
df_best=df.select("id", "experience_level", "biggest_salary", "employee_residence").filter(col("salary_in_usd")==col("biggest_salary"))

print df_best

In [ ]:
df_best.toPandas()

drop duplicates if exist by experience_level

In [ ]:
df_best=df_best.dropDuplicates(["experience_level"])

print df_best

In [ ]:
df_best.show()

create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [ ]:
df_new=df_best.select("experience_level","biggest_salary","employee_residence")
df_new_best=df_new.withColumn("experience_level", when(df_new.experience_level == "MI","middle")
                                 .when(df_new.experience_level == "SE","senior")
                                 .otherwise("Null"))

print df_new_best

In [ ]:
df_new_best.toPandas()

write df_new_best like 1.csv and load then it to df_final

In [ ]:
df_new_best.write.options(header='True', delimiter=',') \
 .csv("C:/Users/user/Downloads/1")
df_final = spark.read.options(header='True',inferSchema='True',delimiter=',') \
.csv("C:/Users/user/Downloads/1.csv")

print df_final

In [ ]:
df_final.show()

filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [ ]:
df_final=df_final.filter(df_final.experience_level != "Null")
df_final=df_final.join(df2, (df_final.biggest_salary ==  df2.salary_in_usd)&(df_final.employee_residence==df2.employee_residence))


print df_final

In [ ]:
df_final.toPandas()

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [ ]:
big_sel=df_final.select(max("biggest_salary")).show()

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system